## NLP03 
- 노드16 : 텍스트의 분포로 벡터화하기
- Word2Vec 같은 임베딩을 활용한 분산 표현을 학습하는 기법이 대중화되기 이전에 많이 쓰이던 방법들

### 노드16 : 텍스트의 분포 벡터화

#### 16-02 단어 빈도(1) - BoW
- Bag of Words는 자연어처리(Natural language processing)이나 정보검색(information Retrieval)에서   
  쓰이는 방법으로 문서 내의 단어 분포를 보고 문서 특성 파악하는 방법
- Bag of Words는 단어들의 순서는 무시(빈도를 그대로)

#### 16-03 단어 빈도(2) - BoW구현
- keras tokenizer 통해 구현

##### keras Tokenizer 활용

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 빈도를 bow에 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

E0000 00:00:1750123530.940727      58 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750123531.082907      58 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750123532.554705      58 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750123532.554769      58 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750123532.554773      58 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750123532.554776      58 computation_placer.cc:177] computation placer already registered. Please check linka

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


##### scikit-learn CountVectorizer 활용
- 각 단어의 빈도만 출력

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


In [3]:
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

단어장(Vocabulary)의 크기 : 10


#### 16-04 단어 빈도(3) - DTM, cosine유사도
- DTM와 코사인 유사도
- DTM은 각 문서에 등장한 단어의 빈도수를 하나의 행렬로 통합
- '행렬'에서 행은 단어, 열은 문서로 표현
  - 문서 숫자 많아지면 자연히 0인 값이 많아짐
- Cosine 유사도는 각 문서의 내적값을 norm의 곱으로 나눈 것
  - 완전 유사하면 1, 반대방향이면 -1, 독립적이면 0

In [4]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [5]:
print('{:.2f}'.format(cos_sim(doc1, doc2))) #문서1과 문서2의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc1, doc3))) #문서1과 문서3의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc2, doc3))) #문서2과 문서3의 코사인 유사도

0.67
0.67
1.00


#### 16-05 단어 빈도(4) - DTM 한계
- 문서, 단어 숫자가 늘어나면 DTM 행렬 값은 자연히 0의 개수가 늘어남
- 불용어 the가 자주 등장한다해서 유사한 문장일까?

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


#### 16-06 단어 빈도(5) TF IDF
- Term Frequency Inverse Document Frequency 의 약자(단어 빈도 역문서 빈도)
- TF-IDF는 불용어처럼 중요도가 낮으면서 모든 문서에 등장하는 단어들이 노이즈가 되는 것을 완화
  - 주의할 점은 TF-IDF를 사용하는 것이 DTM을 사용하는 것보다 성능이 항상 뛰어나지는 않다는 점
  - DTM을 만든 뒤에 TF-IDF 가중치를 DTM에 적용
  
$$
w_{x,y} = \text{tf}_{x,y} \times \log\left(\frac{N}{\text{df}_x}\right)
$$

#### 16-07 단어 빈도(6) TF IDF 구현
- python에서 구현해보기

##### TF-IDF 구현하기

In [7]:
from math import log
import pandas as pd
print('=3')

=3


In [8]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]
print('=3')

=3


In [9]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [10]:
N = len(docs) # 총 문서의 수
N

3

In [11]:
def tf(t,d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc    
    return log(N/(df + 1)) + 1

def tfidf(t, d):
    return tf(t,d)* idf(t)

###### TF함수 사용하여 DTM만들기

In [13]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tf(t, d))
        
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


###### 각 단어의 IDF 구하기

In [14]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


##### scikit-learn TFidVectorizer 활용

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',        
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


#### 16-08 LSA와LDA(1) - LSA
- DTM, TF-IDF는 단어의 의미를 벡터로 표현하지 못한다는 한계
- LSA(Latent Semantic Aanlysis) 잠재 의미 분석
  - 특잇값 분해(Singular Value Decompotion) 에 대해 이해할 필요
  - 특잇값 가운데 가장 큰(다시 말해, 가장 중요한) t개만 남기고 해당 특잇값에 대응되는    
    특이 벡터(singular vector) 들로 행렬 A를 근사(approximate) 하도록 하면,    
    이를 절단된 특잇값 분해(Truncated SVD) 라고 함
- 벡터의 차원을 저차원으로 축소하면서 잠재된 의미를 끌어내는 여러 가지 방법은   
  '워드 임베딩'을 배울 때 더 자세히 다루게 됨


#### 16-09 LSA와 LDA(2) - LSA실습

In [19]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
print('=3')

=3


In [20]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### 데이터 다운로드 및 확인

In [21]:
import os

csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('/aiffel/aiffel/topic_modelling/data/abcnews-date-text.csv',
 <http.client.HTTPMessage at 0x7546946350d0>)

In [22]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

In [23]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [24]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [25]:
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

In [26]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

##### 데이터 정제 및 정규화

In [27]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [28]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


##### 역토큰화 및 DTM 생성

In [29]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc
print('=3')

=3


In [30]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [31]:
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)
print('=3')

=3


In [32]:
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


##### scikit-learn TruncatedSVD 활용

In [33]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 0.01203025, -0.0037377 ,  0.01829498, ...,  0.00317952,
        -0.00164143,  0.01246931],
       [ 0.02903222, -0.01087135,  0.01812564, ..., -0.00208543,
         0.01110092, -0.00245481],
       [ 0.00502664, -0.00206105,  0.00976834, ..., -0.00356178,
        -0.00263444,  0.00250823],
       ...,
       [ 0.02969931,  0.00403697,  0.0250209 , ...,  0.02946473,
        -0.01388232,  0.02399477],
       [ 0.06157543, -0.00496141,  0.13744438, ...,  0.93671305,
        -0.71790373, -0.40744649],
       [ 0.07142386,  0.02905237,  0.00097265, ...,  0.02643367,
        -0.01539836, -0.01594036]])

In [34]:
print(lsa_model.components_.shape)

(10, 5000)


In [35]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어 저장!

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74638), ('man', 0.45351), ('charge', 0.21093), ('new', 0.14093), ('court', 0.11141)]
Topic 2: [('man', 0.69372), ('charge', 0.30116), ('court', 0.16727), ('face', 0.11484), ('murder', 0.10675)]
Topic 3: [('new', 0.83679), ('plan', 0.23655), ('say', 0.18273), ('council', 0.10917), ('govt', 0.10856)]
Topic 4: [('say', 0.73949), ('plan', 0.358), ('govt', 0.16716), ('council', 0.13018), ('urge', 0.07634)]
Topic 5: [('plan', 0.73392), ('council', 0.18244), ('govt', 0.14216), ('urge', 0.0822), ('water', 0.06579)]
Topic 6: [('govt', 0.51951), ('court', 0.27039), ('urge', 0.24201), ('fund', 0.20985), ('face', 0.16557)]
Topic 7: [('charge', 0.53216), ('court', 0.44785), ('face', 0.33478), ('murder', 0.128), ('plan', 0.12325)]
Topic 8: [('win', 0.66951), ('court', 0.28358), ('kill', 0.23794), ('crash', 0.16341), ('council', 0.10959)]
Topic 9: [('court', 0.56866), ('kill', 0.17579), ('face', 0.11755), ('accuse', 0.11458), ('tell', 0.07813)]
Topic 10: [('council', 0.52463), 

#### 16-10 LSA와 LDA(3) LDA
- 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA) 은 토픽 모델링의 또 다른 대표적인 알고리즘
- LDA는 문서들이 토픽들의 혼합으로 구성되어 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정
- LDA 진행과정 요약
  - 사용자는 알고리즘에 토픽의 개수(k) 지정
  - 모든 단어를 k개 중 하나의 토픽에 할당
  - 각 문서의 단어 w는 잘못된 토픽에 할당되어 있으나, 다른 단어들은 옳은 토픽에   
    할당되었다고 가정
    - p(topic t | document d) : 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
    - p(word w | topic t) : 각 토픽들 t에서 해당 단어 w의 분포
- LSA vs LDA 차이
  - LSA는 DTM을 차원 축소하여, 근접 차원에서 토픽 묶는 것
  - LDA는 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출

#### 16-11 LSA와 LDA(4) LDA실습
- LDA는 DTM 또는 TF-IDF를 입력으로 받을 수 있음. 여기서는 TF-IDF를 사용

##### TF-IDF 행렬 생성

In [36]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


##### scikit-learn LDA Model 활용

In [37]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.17024867, 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.0366096 , 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26687206, 0.02914502, 0.02914502, ..., 0.13007484, 0.02916018,
        0.28739608],
       [0.10378115, 0.02637829, 0.12325014, ..., 0.02637829, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.2255442 , ..., 0.03376055, 0.03376055,
        0.03376055]])

In [38]:
print(lda_model.components_.shape)

(10, 5000)


In [41]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합시다
terms = tfidf_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)

Topic 1: [('australia', 9359.06334), ('sydney', 5854.97288), ('attack', 4784.76322), ('change', 4193.63035), ('year', 3924.88997)]
Topic 2: [('government', 6344.07413), ('charge', 5947.12292), ('man', 4519.7974), ('state', 3658.16422), ('live', 3625.10473)]
Topic 3: [('australian', 7666.65651), ('say', 7561.01807), ('police', 5513.22932), ('home', 4048.38409), ('report', 3796.04446)]
Topic 4: [('melbourne', 5298.35047), ('south', 4844.59835), ('death', 4281.78433), ('china', 3214.44581), ('women', 3029.28443)]
Topic 5: [('win', 5704.0914), ('canberra', 4322.0963), ('die', 4025.63057), ('open', 3771.65243), ('warn', 3577.47151)]
Topic 6: [('court', 5246.3124), ('world', 4536.86331), ('country', 4166.34794), ('woman', 3983.97748), ('crash', 3793.50267)]
Topic 7: [('election', 5418.5038), ('adelaide', 4864.95604), ('house', 4478.6135), ('school', 3966.82676), ('2016', 3955.11155)]
Topic 8: [('trump', 8189.58575), ('new', 6625.2724), ('north', 3705.40987), ('rural', 3521.42659), ('donald',

#### 16-12 형태소 분석기와 단어 미등록 문제
- 텍스트 분포를 이용한 비지도 학습 토크나이저-1

##### 형태소 분석기 필요성
- 한국어는 교착어
- 교착어란 하나의 낱말(엄밀히는 하나의 어절)이 하나의 어근(root)(혹은 어간(stem))과    
  각각 단일한 기능을 가지는 하나 이상의 접사(affix)의 결합으로 이루어져 있는 언어
- 영어처럼 단순 띄어쓰기로 토큰화가 수행안됨

In [42]:
en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


In [43]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


In [44]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


In [45]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


#### 16-13 soynlp
- 텍스트 분포를 이용한 비지도 학습 토크나이저-2
- soynlp는 품사 태깅, 형태소 분석 등을 지원하는 한국어 형태소 분석기
- 비지도 학습으로 형태소 분석을 한다는 특징
- 데이터에 자주 등장하는 단어들을 형태소로 분석
- soynlp 형태소 분석기는 내부적으로 단어 점수표로 동작
  - 응집 확률(cohesion probability) 과 브랜칭 엔트로피(branching entropy) 를 활용

##### soynlp 실습

In [46]:
import urllib.request

txt_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('/aiffel/aiffel/topic_modelling/data/2016-10-20.txt',
 <http.client.HTTPMessage at 0x7545fbdf57c0>)

In [47]:
from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

In [48]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [49]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.927 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


##### soynlp의 응집 확률(cohesion probability)

$$
\text{cohesion}(n) = \left( \prod_{i=1}^{n-1} P(c_{1:i+1} \mid c_{1:i}) \right)^{\frac{1}{n-1}}
$$


$\text{cohesion}(2) = P(\text{반포} \mid \text{반})\$

$\text{cohesion}(3) = \sqrt[2]{P(\text{반포} \mid \text{반}) \cdot P(\text{반포한} \mid \text{반포})} \$

$\text{cohesion}(4) = \sqrt[3]{P(\text{반포} \mid \text{반}) \cdot P(\text{반포한} \mid \text{반포}) \cdot P(\text{반포한강} \mid \text{반포한})} \$

$ \text{cohesion}(5) = \sqrt[4]{P(\text{반포} \mid \text{반}) \cdot P(\text{반포한} \mid \text{반포}) \cdot P(\text{반포한강} \mid \text{반포한}) \cdot P(\text{반포한강공} \mid \text{반포한강})} \$

$ \text{cohesion}(6) = \sqrt[5]{P(\text{반포} \mid \text{반}) \cdot P(\text{반포한} \mid \text{반포}) \cdot P(\text{반포한강} \mid \text{반포한}) \cdot P(\text{반포한강공} \mid \text{반포한강}) \cdot P(\text{반포한강공원} \mid \text{반포한강공})} \$

$ \text{cohesion}(7) = \sqrt[6]{P(\text{반포} \mid \text{반}) \cdot P(\text{반포한} \mid \text{반포}) \cdot \cdots \cdot P(\text{반포한강공원에} \mid \text{반포한강공원})} \$

In [50]:
word_score_table["반포한"].cohesion_forward

0.08838002913645132

In [51]:
word_score_table["반포한강"].cohesion_forward

0.19841268168224552

In [52]:
word_score_table["반포한강공"].cohesion_forward

0.2972877884078849

In [53]:
word_score_table["반포한강공원"].cohesion_forward

0.37891487632839754

In [56]:
word_score_table["반포한강공원에"].cohesion_forward

0.33492963377557666

##### soynlp의 브랜칭 엔트로피(branching entropy)
- 브랜칭 엔트로피(Branching Entropy) 는 확률 분포의 엔트로피값을 사용

In [57]:
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [58]:
word_score_table["디스플"].right_branching_entropy

-0.0

In [59]:
word_score_table["디스플레"].right_branching_entropy

-0.0

In [61]:
# 하나의 단어가 끝나면 그 경계 부분부터 다시 브랜칭 엔트로피값이 증가
word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

##### soynlp의 LTokenizer
- 띄어쓰기 단위로 잘 나뉜 문장은 L 토크나이저(LTokenizer)를 사용
- 한국어는 띄어쓰기 단위로 나눈 어절 토큰이 주로 L 토큰 + R 토큰의 형식

In [62]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

##### 최대 점수 토크나이저
- 최대 점수 토크나이저(MaxScoreTokenizer)는 띄어쓰기가 되어 있지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저

In [63]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']